In [1]:
import os
import datetime
import json
import twython
import json

In [2]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
import nltk
from nltk.tokenize import word_tokenize

In [3]:
import pandas as pd
import numpy as np 

In [107]:
def reset(df):
    """
    pass in dataframe and returns dataframe with index reset
    just wanted to make it a bit of a quicker type since I'm using it so much 
    """
    return df.reset_index(drop=True, inplace=True)

def remove_tweets(df, column, list_to_remove):
    """
    Pass in your dataframe, a column & a list of words to search for and remove.
    Does not change capitalization, but will remove if word is within another word.
    Returns dataframe with words removed. 
    """
    for item in list_to_remove:
        reset(df)
        for i in range(len(df)):
            if type(df.at[i, column]) == str:
                if item in df.at[i, column]:
                    df.drop(index=i, inplace=True)
    return df 

def remove_phrase(df, column, dict_to_remove):
    """
    Pass in your dataframe, a column & a dictionary with lowercase key value pairs of words to search for and remove.
    Will change capitalization & remove if words are within other words.
    Returns dataframe with words removed. 
    """
    for key in dict_to_remove.keys():
        reset(df)
        value = dict_to_remove[key].lower()
        key = key.lower()
        for i in range(len(df)):
            if type(df.at[i, column]) == str:
                if key in df.at[i, column] and value in df.at[i, column]:
                    df.drop(index=i, inplace=True)
    return df 

In [21]:
df = pd.read_csv('tw_319_nd')
df = df.append(pd.read_csv('beeep'))

In [22]:
df = df.drop(['Unnamed: 0', 'level_0', 'index', 'Unnamed: 0.1', 'Unnamed: 1'], 1)
reset(df)

In [23]:
# since every time we loop through it starts a new index, there are now multiple entries for each index, we need to fix this
# and the easiest way is to just reset the index once it finishes running 
df = df.drop_duplicates(subset='post_id')
reset(df)

In [25]:
# in order to add our strings for our hashtags and mentions together we had to make the entire column a string, sinlucding empty cells
# here we go through and replace empty cells with nan values so pandas will read them as being empty instead of a string
for i in range(len(df)):
    if df.at[i, 'mentions'] == ' ':
        df.at[i, 'mentions'] = np.nan
    else: 
        pass 
    if df.at[i, 'hashtag'] == ' ':
        df.at[i, 'hashtag'] = np.nan
    else: 
        pass 

In [26]:
df = df.sort_values(by='favorite_count', ascending=False).drop_duplicates(subset='text', keep='first')
reset(df)
df['text'] = df['text'].str.lower()

In [27]:
for i in range(len(df)):
    df.at[i, 'text'] = df.at[i, 'text'].replace('RT', '')

In [134]:
hashtags = ['WandaVision', 'findyourthing', 'gangsters', 'RBandME', 'music', 'homework', 'Termpaper', 'esa', 'newmusic', 
            'nowplaying', 'fridaylivestream',  'BTSpace', 'CRAVITY',  'Bitcoin', 'StanWorld', 'BecomeOneForIZONE', 'ovni', 
            'Colchester', 'NowPlaying', 'IZONE_PERMANENT', 'Onlineclass', 'WorldBookDay', 'SatyamevaJayate2', 'izone_permanent',
            'SnyderCut', '마마무', 'MAMAMOO',  'EVA71', '하게', 'OurParallelUniverseContinues', 'جیو_تو_عیسیٰ_کی_طرح', 'dogecoin', 
            'ShowtimeBetAngMalupet', 'DidYouKnow', 'VoteHarryStyles', 'AMN𝗘𝗦𝗜𝗔',  'GRAMMYs', 'etsy', 'MyanmarMilitaryTerrorists',
            'Poshmark', 'StPatricksDay', 'MindBreeze', 'ad', 'ArtOfTheBlue', 'मुस्लिमभाई_सुनोअल्लाहकीसच्चाई', 'Essaydue', 'BigData', 
            'Aylesbury',  'PiDay', 'Harpenden',  'SoundCloud', 'Dogecoin', 'doge',  'عمران_نیازی_گٹر_کا_کیڑا', 'ZackSnydersJusticeLeague',
            'NFT',  'ifttt']

users = ['artemis_twt']

text = ['esa_celebnews']

dict_to_remove = {'bruno': 'mars', 'space':'jam'}

In [135]:
dict_of_lists = {'user' : users, 'hashtag' : hashtags, 'text': text}

for key in dict_of_lists:
    df = remove_tweets(df, key, dict_of_lists[key])

In [136]:
# df = remove_phrase(df, 'text', dict_to_remove)
len(df)

358422

In [137]:
count = 0 
empty_list = []

for item in df.hashtag.value_counts().to_frame().reset_index()['index']:
    if count < 250:
        hashtags = str(item).split()
        for item in hashtags:
            empty_list.append(item)
            count = count + 1

items = ['jaxa', 'esa', 'curiosityrover', 'areospace', 'internationalspacestation', 'JAXA', 'astronomy',
         'oppertunityrover', 'virgingalactic', 'universe', 'sls', 'Starship', 'climate', 'starship', 'virginorbit', 
         'nasa', 'cosmos', 'mars', 'falconheavy', 'NASA', 'futurism', 'starliner', 'iss', 'spacex', 'falcon9', 
         'nasa_app', 'roscosmos', 'Roscosmos', 'blueorigin', 'ESA', 'spacetravel', 'artemis', 'marswebcam', 'starlink']

fresh_hashtags = []

for item in empty_list:
    if item not in items:
        fresh_hashtags.append(item)
        
print(set(fresh_hashtags))

{'StarWalk', 'ElonMusk', 'InternationalWomensDay', 'BREAKING', 'ISSoverDaisenTottoriJP', 'Astronomy', 'FridayLivestream', 'SPACE', 'think', 'dearMoonCrew', 'dearmooncrew', 'EU', 'luna', 'energystorage', 'ParkerSolarProbe', 'Perseverance', 'Nasa', 'Starlink', 'space', 'industry', 'Falcon9', 'SN10', 'BritishScienceWeek', 'STEM', 'Challenger', 'USA', 'ISS', 'AI', 'TournamentEarth', 'science', 'AskNASA', 'job', 'WomenInSTEM', 'DYK', 'Cosmos', 'SmartNews', 'Webb', 'SpaceXFleet', 'Wallpaper', 'Technology', 'Venus', 'copper', 'Quantum', 'News', 'rehearsals', 'meeting', 'skywatching', 'LPSC2021', 'StarTrek', 'astrophotography', 'KeepLookingUp', 'spaceflight', 'StormHour', 'OTD', 'Cosmology', 'SpaceX', 'Myanmar', 'MissionAlpha', 'Futurism', 'Aerospace', 'denver', 'IWD2021', 'apod', 'F1', 'SpaceHour', 'Marriage', 'hindi', 'VirginGalactic', 'Sentinel', 'NASA_App', '67P', 'Spacex', 'PerseveranceRover', 'YoutubeIndia', 'WomensHistoryMonth', 'OnThisDay', 'InternationalSpaceStation', 'COVID19', 'Russ

In [28]:
# def tokenize(text):
#     tokenized = word_tokenize(text)
#     return tokenized

# for i in range(len(df)):
#     df.at[i, 'text'] = tokenize(df.at[i, 'text'])

In [17]:
df = pd.read_csv('yt')
print(len(df))
df = df.append(pd.read_csv('yt1'))
print(len(df))
df = df.append(pd.read_csv('yt2'))
print(len(df))

164467
202489
226925


In [18]:
df.to_csv('yt0')

In [25]:
all_words = ''

# for i in range(len(df)):
#     word_list = df.at[i, 'text']
#     for word in word_list:
#         all_words = all_words + '' + word

In [26]:
# list_ = all_words.split(' ')

In [27]:
# dict_ = {'item': [], 'count':[]}
# for item in list_:
#     dict_['item'].append(item)
#     dict_['count'].append(list_.count(item))